In [ ]:
# Import necessary libraries
import json
import pandas as pd
import gspread
from kaggle_secrets import UserSecretsClient
from datetime import datetime, timedelta, date
import sys

print("Authenticating with Google...")
try:
    user_secrets = UserSecretsClient()
    google_creds_json = user_secrets.get_secret("GOOGLE_SHEETS_CREDENTIALS")
    google_creds_dict = json.loads(google_creds_json)
    gc = gspread.service_account_from_dict(google_creds_dict)
    print("Authentication successful!")
except Exception as e:
    print(f"Authentication failed: {e}")
    raise

spreadsheet = gc.open_by_url(
    'https://docs.google.com/spreadsheets/d/1yekpw2BcsAGxU--PDlPUDl4W2BKyBKH2XBYpmkqp_bI/edit#gid=0'
)

# Force a fresh pull of spreadsheet metadata (from the server)
metadata = spreadsheet.fetch_sheet_metadata()

# You can inspect it if you want:
print("Spreadsheet title:", metadata["properties"]["title"])
print("Last modified time:", metadata["properties"].get("modifiedTime", "unknown"))

# Then explicitly reload your worksheet and its contents:
worksheet = spreadsheet.get_worksheet(0)
rows = worksheet.get_all_values()  # <-- this always gets the *live* data
print(f"✅ Loaded {len(rows)} rows from live sheet")
print("Sample last row:", rows[-1])

In [ ]:
# --- Get subjects woorksheet ---

worksheet = spreadsheet.get_worksheet(0)
data = worksheet.get_all_records()
df = pd.DataFrame(data)

if 'Date max' in df.columns and 'Traited' in df.columns:
    df['Date max'] = pd.to_datetime(df['Date max'], format='%d/%m/%Y')
    df['Traited'] = df['Traited'].astype(str).str.strip().str.upper() == 'TRUE'
    today = pd.to_datetime('today').normalize()

    # The 'Traited' column is now a proper boolean type.
    # Filter the DataFrame based on the conditions:
    # 1. 'Date max' is in the future
    # 2. 'Traited' is False
    filtered_df = df[(df['Date max'] > today) | (df['Traited'] == False)]

    print("\nFiltered data (where 'Date max' > today OR 'Traited' is False):")
    if not filtered_df.empty:
        print(filtered_df)
    else:
        print("No rows match the specified criteria.")
        print("Stopping Kaggle notebook execution.")
        sys.exit()



# This cell assumes 'gspread' is imported, as gspread.Cell is used.
# If not already imported, add: import gspread

print("\nChecking for overdue and untraited items to update in Google Sheet...")

# 1. Define the condition: 'Date max' < today AND 'Traited' == False
condition = (df['Date max'] < today) & (df['Traited'] == False)

# 2. Get the subset of the DataFrame that matches this condition
rows_to_update = df[condition]

if not rows_to_update.empty:
    print(f"Found {len(rows_to_update)} overdue item(s) to mark as 'TRUE'.")
    
    try:
        # 3. Get the column number for 'Traited'
        # We add 1 because gspread columns are 1-indexed, while df columns are 0-indexed
        traited_col_index = df.columns.get_loc('Traited') + 1
        
        # 4. Prepare a list of cells for batch update
        cells_to_update = []
        for df_index in rows_to_update.index:
            # df.index 0 corresponds to sheet row 2 (after header)
            sheet_row = int(df_index) + 2 
            
            # Create a gspread.Cell object with the row, col, and new value
            cells_to_update.append(gspread.Cell(sheet_row, traited_col_index, True))
        
        # 5. Perform the batch update in the Google Sheet
        if cells_to_update:
            worksheet.update_cells(cells_to_update)
            print(f"Successfully updated {len(cells_to_update)} cells in the Google Sheet.")
            
            # 6. (Optional) Update the local DataFrame to reflect this change
            df.loc[condition, 'Traited'] = True
            print("Local DataFrame 'df' has been updated to match.")

    except ValueError:
        print("Error: 'Traited' column not found in the DataFrame.")
    except Exception as e:
        print(f"An error occurred during the Google Sheet update: {e}")

else:
    print("No overdue and untraited rows found. No updates were needed.")

In [ ]:
# Convert a (filtered) Google Sheet DataFrame into the `subjects` JSON you specified.
# Fix: month number now correctly written in dates (no more "m" placeholders).

import pandas as pd
import re
import json
from datetime import datetime

# ---------- Helpers ----------
def canonize(col: str) -> str:
    """Lowercase, collapse internal whitespace to single space, strip ends."""
    return re.sub(r"\s+", " ", str(col)).strip().lower()

def to_iso_z(val):
    """Convert many date representations to 'YYYY-MM-DDT00:00:00Z' or return None."""
    if val is None:
        return None
    if isinstance(val, pd.Timestamp):
        if pd.isna(val):
            return None
        return val.strftime("%Y-%m-%dT00:00:00Z")

    s = str(val).strip()
    if not s or s.lower() in {"nan", "nat"}:
        return None

    # Handle keyword TODAY
    if s.lower() == "today":
        return datetime.now().strftime("%Y-%m-%dT00:00:00Z")

    # Handle numeric or string month-day-year issues (replace accidental 'm' placeholders)
    s = re.sub(r"[^0-9/\-]", "", s)

    # Try common explicit formats
    for fmt in ("%d/%m/%Y", "%Y-%m-%d", "%m/%d/%Y", "%d-%m-%Y", "%Y/%m/%d"):
        try:
            dt = datetime.strptime(s, fmt)
            return dt.strftime("%Y-%m-%dT00:00:00Z")
        except ValueError:
            continue

    # Fallback to pandas parser (dayfirst=True for European dates)
    dt = pd.to_datetime(s, dayfirst=True, errors="coerce")
    if pd.isna(dt):
        return None
    return pd.Timestamp(dt).strftime("%Y-%m-%dT00:00:00Z")

def safe_int(val, default=0):
    """Safely convert a value to an integer, returning a default on failure."""
    try:
        if pd.isna(val):
            return default
    except Exception:
        pass
    try:
        return int(str(val).strip())
    except (ValueError, TypeError):
        try:
            return int(float(val))
        except (ValueError, TypeError):
            return default

def normalize_headers(df: pd.DataFrame) -> pd.DataFrame:
    """Create a copy of the DataFrame with canonical header names."""
    df = df.copy()
    df.columns = [canonize(c) for c in df.columns]
    return df

# ---------- Acquire base_df ----------
if "filtered_df" in globals():
    base_df = filtered_df.copy()
else:
    if "df" not in globals():
        raise RuntimeError("Neither 'filtered_df' nor 'df' exist. Run the previous cell that builds them.")
    df_norm = normalize_headers(df)
    if "date max" not in df_norm.columns or "traited" not in df_norm.columns:
        raise RuntimeError("Expected columns 'Date max' and 'Traited' not found (even after header normalization).")

    try:
        df_norm["date max"] = pd.to_datetime(df_norm["date max"], format="%d/%m/%Y", errors="coerce")
    except Exception:
        df_norm["date max"] = pd.to_datetime(df_norm["date max"], errors="coerce", dayfirst=True)

    df_norm["traited"] = (
        df_norm["traited"]
        .astype(str)
        .str.strip()
        .str.upper()
        .isin(["TRUE", "1", "YES", "Y", "T"])
    )

    today = pd.to_datetime("today").normalize()
    mask = (df_norm["date max"] > today) | (df_norm["traited"] == False)
    base_df = df_norm.loc[mask].copy()

# ---------- Build subjects JSON ----------
base_df = normalize_headers(base_df)
canon_cols = list(base_df.columns)

category_cols = [c for c in canon_cols if c.startswith("category")]
category_cols = sorted(category_cols, key=lambda c: int(re.sub(r"\D", "", c) or 0))

subjects = []
skipped_rows_log = []

for index, row in base_df.iterrows():
    topic = str(row.get("subject description", "")).strip()
    if not topic:
        skipped_rows_log.append(f"GSheet Row {index + 2}: Skipped because 'Subject Description' was empty.")
        continue

    cats = []
    for idx, col in enumerate(category_cols, start=1):
        val = row.get(col, None)
        name = str(val).strip() if pd.notna(val) else ""
        if name:
            cats.append({"id": idx, "name": name})

    nb_research_dev = safe_int(row.get("number of research variation", 0))
    max_results = safe_int(row.get("number of url to check per research", 0))
    max_final_results = safe_int(row.get("number of desired results", 0))
    start_val = row.get("start research date", None)
    end_val = row.get("end research", None)

    subjects.append({
        "topic": topic,
        "categories": cats,
        "nb of research deviations": nb_research_dev,
        "max_results": max_results,
        "max_final_results": max_final_results,
        "nb of results": 0,
        "date research start": to_iso_z(start_val),
        "date research end": to_iso_z(end_val),
    })

payload = {"subjects": subjects}
json_str = json.dumps(payload, ensure_ascii=False, indent=2)

output_path = "/kaggle/working/subjects.json"
with open(output_path, "w", encoding="utf-8") as f:
    f.write(json_str)

print("--- Generated JSON ---")
print(json_str)
print(f"\n✅ Successfully saved JSON to {output_path}")

if skipped_rows_log:
    print("\n--- Validation Summary ---")
    for log_entry in skipped_rows_log:
        print(log_entry)
else:
    print("\n--- Validation Summary ---")
    print("All processed rows were valid.")


In [ ]:
# --- Get settings ---
from datetime import datetime, timedelta, time
import sys

worksheet = spreadsheet.get_worksheet(1)
data = worksheet.get_all_records()
df = pd.DataFrame(data)

# Read settings
try:
    RECURENCE = int(str(df['Recurence'][0]).strip())
except Exception:
    raise ValueError("Recurence must be an integer number of days.")

NEXT_NEWSLETTER_DATE_STR = str(df['Next date'][0]).strip()
EXCLUDED_DAYS_STR = str(df.get('Excluded days', [''])[0]).strip()

def parse_ddmmyyyy_or_y(date_str: str) -> datetime:
    """Parse DD/MM/YYYY or DD/MM/YY to a datetime (00:00)."""
    for fmt in ("%d/%m/%Y", "%d/%m/%y"):
        try:
            return datetime.strptime(date_str, fmt)
        except ValueError:
            continue
    # Last resort: pandas
    try:
        import pandas as pd
        dt = pd.to_datetime(date_str, dayfirst=True, errors="raise")
        return datetime(dt.year, dt.month, dt.day)
    except Exception:
        raise ValueError(f"Next date '{date_str}' must be in format DD/MM/YYYY or DD/MM/YY.")

# Parse next date (sheet stores DD/MM/YYYY or DD/MM/YY)
next_date = parse_ddmmyyyy_or_y(NEXT_NEWSLETTER_DATE_STR)

# Excluded weekdays
weekday_map = {
    'monday': 0, 'tuesday': 1, 'wednesday': 2, 'thursday': 3,
    'friday': 4, 'saturday': 5, 'sunday': 6
}
excluded_day_names = [d.strip().lower() for d in EXCLUDED_DAYS_STR.split(',') if d.strip()]
excluded_weekdays = {weekday_map[d] for d in excluded_day_names if d in weekday_map}

today = datetime.now().date()

# ---- Recurrence roll-forward logic ----
# Keep adding recurrence until the date is >= today.
while next_date.date() < today:
    next_date += timedelta(days=RECURENCE)

# If we landed in the future, only then skip excluded weekdays (do NOT skip if it's exactly today).
if next_date.date() > today:
    while next_date.weekday() in excluded_weekdays:
        next_date += timedelta(days=1)

# Update the "Next date" in the Google Sheet to the adjusted value (DD/MM/YYYY)
headers = worksheet.row_values(1)
try:
    col_next_date = headers.index('Next date') + 1
except ValueError:
    raise ValueError("Column 'Next date' not found in settings sheet header row.")

new_date_str = next_date.strftime('%d/%m/%Y')
worksheet.update_cell(2, col_next_date, new_date_str)

# Continue only if the (possibly adjusted) date is today
if next_date.date() != today:
    print(f"\n🔴 Newsletter date ({new_date_str}) is not today ({today.strftime('%d/%m/%Y')}).")
    print("Stopping Kaggle notebook execution.")
    sys.exit()
else:
    print(f"\n🟢 Newsletter date is today ({new_date_str}). Continuing…")
    NEXT_NEWSLETTER_DATE = next_date


In [ ]:
worksheet = spreadsheet.get_worksheet(2) # To get the third tab

# --- Fetch all data from the worksheet ---
print("Fetching all records from the worksheet...")
data = worksheet.get_all_records()

# --- Convert to a pandas DataFrame ---
if data:
    links_df = pd.DataFrame(data)
    print("\nSuccessfully fetched data. Here's a preview:")
    print(links_df.head()) # Display the first 5 rows

In [ ]:
!pip install --no-cache-dir --force-reinstall \
  --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu121 \
  "llama-cpp-python>=0.3.8"

In [ ]:
# install the maintained client
!pip -q install -U ddgs

In [ ]:
# -------------------------------------------
# 2.5️⃣  Llama.cpp (Gemma-3) GPU init for DeepSearch
#       - uses both T4s with tensor_split
#       - deterministic generation for JSON-only outputs
# -------------------------------------------
import os
from llama_cpp import Llama
import json

# Use cuBLAS path if available (set BEFORE importing/instantiating in a fresh kernel)
os.environ.setdefault("GGML_CUDA_FORCE_MMQ", "0")
os.environ.setdefault("GGML_CUDA_FORCE_CUBLAS", "1")
os.environ.setdefault("LLAMA_LOG_LEVEL", "WARN")   # set to INFO for detailed logs
# Target both T4s (optional)
os.environ.setdefault("CUDA_VISIBLE_DEVICES", "0,1")

LLAMA_MODEL_PATH = "/kaggle/input/gemma-3/gguf/gemma-3-12b-it-qat-q4_0/3/gemma-3-12b-it-q4_0.gguf"

# Create a single global instance reused by the whole pipeline
llm = Llama(
    model_path=LLAMA_MODEL_PATH,
    n_gpu_layers=-1,
    tensor_split=[0.5, 0.5],
    main_gpu=0,
    offload_kqv=True,
    n_ctx=4096,          # ⬅️ bump from 2048 → 4096 (or 6144/8192 if you have VRAM)
    n_batch=256,         # ⬅️ speeds prompt ingestion (lower if you OOM)
    verbose=False,
)

# Small helper to enforce "JSON only" and deterministic decoding
def _chat_json_only(messages, max_new_tokens=512):
    """
    messages: List[{'role': 'system'|'user'|'assistant', 'content': str}]
    returns a str (model content)
    """
    # Make sure there is a strong system instruction
    sys_prefix = {"role": "system", "content": "You only output JSON — no extra text."}
    msgs = [sys_prefix] + [m for m in messages if m.get("role") != "system"]

    out = llm.create_chat_completion(
        messages=msgs,
        temperature=0.0,
        top_p=1.0,
        seed=12345,                # deterministic
        max_tokens=max_new_tokens,
        # stop can be omitted; JSON extraction below is robust
    )
    return out["choices"][0]["message"]["content"]

In [ ]:
# -------------------------------------------
# 2️⃣ Subjects (Structured Metadata)
#    - User provides dates manually (we DO NOT overwrite them)
#    - "nb of results" = **max_results** used for DDG (per subject)
#    - "categories" = list of categories to classify (ids + labels)
#    - "max_final_results" = max items to keep per subject, sorted by relevance

with open("/kaggle/working/subjects.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# If it's a dict with numeric keys, convert values to list
if isinstance(data, dict):
    # Sort keys numerically if possible
    try:
        subjects = [v for k, v in sorted(data.items(), key=lambda x: int(x[0]))]
    except Exception:
        subjects = list(data.values())
else:
    subjects = data

if isinstance(subjects, list) and len(subjects) == 1 and isinstance(subjects[0], list):
    subjects = subjects[0]

# If it's a dict with numeric keys -> turn into a list (keeps numeric order)
elif isinstance(subjects, dict):
    try:
        subjects = [v for k, v in sorted(subjects.items(), key=lambda kv: int(kv[0]))]
    except Exception:
        subjects = list(subjects.values())

# If it's a list of JSON strings -> parse each string
elif isinstance(subjects, list) and all(isinstance(x, str) for x in subjects):
    subjects = [json.loads(x) for x in subjects]

print(f"✅ Normalized to {len(subjects)} subjects. Example keys:", list(subjects[0].keys()))


# -------------------------------------------
# 3️⃣ Imports & Utility Functions
# -------------------------------------------
import json, re, time, gc
from typing import Any, Dict, List, Optional
import pandas as pd
from tqdm.auto import tqdm
from ddgs import DDGS
import torch

# NOTE: These are assumed to be available elsewhere in the notebook:
# - tok, textgen, eos_id (for chat template inference)
# - torch (for cuda empty cache)
# - DDGS from duckduckgo_search (for web search)

# ---- Web metadata fetch helpers ----
import requests, urllib.parse as urlparse
from bs4 import BeautifulSoup

def _abs_url(base, href):
    try:
        return urlparse.urljoin(base, href)
    except Exception:
        return None

def _same_domain(u1, u2):
    try:
        return urlparse.urlparse(u1).netloc == urlparse.urlparse(u2).netloc
    except Exception:
        return False

def fetch_page_metadata(url: str, max_links: int = 6, timeout: float = 8.0) -> dict:
    """
    Returns a compact dict describing the page, or {} on failure.
    Avoids heavy downloads; times out quickly; strips long text.
    """
    try:
        resp = requests.get(
            url,
            headers={"User-Agent": "Mozilla/5.0 (compatible; DeepSearch/1.0)"},
            timeout=timeout,
            allow_redirects=True,
        )
    except Exception:
        return {}

    ctype = resp.headers.get("Content-Type", "")
    if "html" not in ctype.lower():
        return {}

    html = resp.text
    soup = BeautifulSoup(html, "html.parser")

    # Title via <title> or og:title
    title = (soup.title.string or "").strip() if soup.title and soup.title.string else ""
    og_title = soup.find("meta", property="og:title")
    if og_title and og_title.get("content"):
        title = og_title.get("content").strip() or title

    # Description via og:description, meta[name=description], or first <p>
    description = ""
    og_desc = soup.find("meta", property="og:description")
    meta_desc = soup.find("meta", attrs={"name": "description"})
    if og_desc and og_desc.get("content"):
        description = og_desc.get("content").strip()
    elif meta_desc and meta_desc.get("content"):
        description = meta_desc.get("content").strip()
    if not description:
        first_p = soup.find("p")
        if first_p:
            description = " ".join(first_p.get_text(" ").split())

    # Canonical URL
    canonical = soup.find("link", rel=lambda v: v and "canonical" in v)
    canonical_url = canonical.get("href").strip() if canonical and canonical.get("href") else ""

    # One H1 as a hint
    h1 = soup.find("h1")
    h1_text = " ".join(h1.get_text(" ").split()) if h1 else ""

    # A few internal links (href, text)
    all_links, seen = [], set()
    for a in soup.find_all("a", href=True):
        href = _abs_url(resp.url, a.get("href"))
        if not href or href in seen:
            continue
        seen.add(href)
        if _same_domain(resp.url, href):
            text = " ".join((a.get_text(" ") or "").split())
            if text:
                all_links.append({"href": href, "text": text[:120]})
        if len(all_links) >= max_links:
            break

    # --- NEW: try to extract publish/modified timestamps from common tags ---
    # --- Safe meta getter: always route through attrs=... to avoid name= collision ---
    def _meta_content(**attrs):
        tag = soup.find("meta", attrs=attrs)
        return tag.get("content").strip() if tag and tag.get("content") else ""

    published_raw = (
        _meta_content(property="article:published_time")
        or _meta_content(name="pubdate")
        or _meta_content(name="publish-date")
        or _meta_content(name="date")
        or _meta_content(itemprop="datePublished")
    )
    
    modified_raw = (
        _meta_content(property="article:modified_time")
        or _meta_content(name="lastmod")
        or _meta_content(itemprop="dateModified")
    )
    
    # Also check <time datetime="...">
    if not published_raw:
        t = soup.find("time", attrs={"datetime": True})
        if t and t.get("datetime"):
            published_raw = t.get("datetime").strip()

    def _trim(s, n=240):
        return (s[:n] + "…") if s and len(s) > n else s

    # Parse dates to ISO strings if possible (actual parsing helpers added below)
    published_at_dt = _to_utc(_parse_dt(published_raw)) if published_raw else None
    modified_at_dt  = _to_utc(_parse_dt(modified_raw))  if modified_raw  else None

    return {
        "url": resp.url or url,
        "canonical": canonical_url or "",
        "title": _trim(title),
        "h1": _trim(h1_text),
        "description": _trim(description, 320),
        "links": all_links,
        "published_at": published_at_dt.isoformat() if published_at_dt else "",
        "modified_at": modified_at_dt.isoformat() if modified_at_dt else "",
    }

# ---- Search & JSON parsing helpers (from original cell) ----
def parse_json_list(text: str) -> Optional[List[str]]:
    if not text:
        return None
    try:
        data = json.loads(text)
        if isinstance(data, list) and all(isinstance(x, str) for x in data):
            return data
    except Exception:
        pass
    m = re.search(r"\[.*?\]", text, flags=re.DOTALL)
    if m:
        try:
            data = json.loads(m.group(0))
            if isinstance(data, list) and all(isinstance(x, str) for x in data):
                return data
        except Exception:
            return None
    return None

# ---
# --- FIX 1: Make parser handle both lists [...] and objects {...}
# ---
def extract_json_object(text: str) -> Optional[Any]: # Return Any (dict or list)
    if not text:
        return None
    txt = text.strip().replace("```json", "").replace("```", "")
    
    # 1. Try to parse the whole string directly
    try:
        data = json.loads(txt)
        if isinstance(data, (dict, list)):
            return data
    except Exception:
        pass # Not a clean JSON, fallback to regex

    # 2. Fallback: Search for the first '[' or '{'
    txt_clean = txt.strip()
    first_char = ""
    for char in txt_clean:
        if char in ['{', '[']:
            first_char = char
            break
            
    m = None
    if first_char == '{':
        m = re.search(r"\{.*\}", txt_clean, flags=re.DOTALL)
    elif first_char == '[':
        m = re.search(r"\[.*\]", txt_clean, flags=re.DOTALL)

    if m:
        try:
            # Try parsing the extracted regex block
            data = json.loads(m.group(0))
            if isinstance(data, (dict, list)): 
                 return data # Return the parsed data (could be dict or list)
        except Exception:
            return None # Failed to parse regex match
            
    return None # Return None if everything fails

# -------------------------------------------
# 4️⃣ Chat Template Wrapper (Gemma-safe via llama.cpp)
# -------------------------------------------
def _to_parts(content):
    """Normalize content to a list of {type: 'text', text: '...'} parts."""
    if isinstance(content, str):
        return [{"type": "text", "text": content}]
    if isinstance(content, list):
        if all(isinstance(x, dict) and "type" in x and "text" in x for x in content):
            return content
        if all(isinstance(x, str) for x in content):
            return [{"type": "text", "text": x} for x in content]
    return [{"type": "text", "text": str(content)}]

def chat_generate_json(messages, max_new_tokens: int = 512) -> str:
    """
    Deterministic JSON-only generation using llama.cpp chat completion.
    Ignores tok/textgen/eos_id; uses the GGUF-embedded chat template.
    """
    # Keep your message normalization (so upstream calls don't change)
    norm_messages = [{"role": m["role"], "content": _to_parts(m.get("content", ""))} for m in messages]

    # Flatten parts back to plain strings for llama.cpp
    flat = []
    for m in norm_messages:
        parts = m["content"]
        text = "\n".join(p.get("text", "") for p in parts if isinstance(p, dict))
        flat.append({"role": m["role"], "content": text})

    return _chat_json_only(flat, max_new_tokens=max_new_tokens)

# -------------------------------------------
# 5️⃣ Date helpers + DDG with date filter
# -------------------------------------------
from datetime import datetime, timezone, timedelta
try:
    from dateutil import parser as dtparse  # more robust parsing if available
except Exception:
    dtparse = None

def _parse_dt(s: str) -> Optional[datetime]:
    if not s:
        return None
    s = s.strip().replace("Z", "+00:00")
    try:
        if dtparse:
            return dtparse.parse(s)
        return datetime.fromisoformat(s)
    except Exception:
        return None

def _to_utc(dt: Optional[datetime]) -> Optional[datetime]:
    if not dt:
        return None
    if dt.tzinfo is None:
        return dt.replace(tzinfo=timezone.utc)
    return dt.astimezone(timezone.utc)

def _pick_timelimit_for_range(start_utc: datetime, end_utc: datetime) -> Optional[str]:
    """Map an exact range to DDG timelimit granularity when it fits neatly."""
    delta = end_utc - start_utc
    if delta <= timedelta(days=1, seconds=1):
        return "d"  # last day
    if delta <= timedelta(days=7, seconds=1):
        return "w"  # last week
    if delta <= timedelta(days=31, seconds=1):
        return "m"  # last month
    if delta <= timedelta(days=365, seconds=1):
        return "y"  # last year
    return None

def _within_range(dt_val: Optional[datetime], start_utc: Optional[datetime], end_utc: Optional[datetime]) -> bool:
    if not dt_val:
        return False
    dt_val = _to_utc(dt_val)
    if start_utc and dt_val < start_utc:
        return False
    if end_utc and dt_val > end_utc:
        return False
    return True

def search_ddg(
    subject_obj: Dict[str, Any],
    query: str,
    max_results: int = 20,
    date_start_iso: Optional[str] = None,
    date_end_iso: Optional[str] = None,
):
    """
    DuckDuckGo search with an in-query date filter only.
    We map the requested window length to DDG's relative timelimit {d,w,m,y}.
    NOTE: DDG timelimit is 'last day/week/month/year' (relative to now), not an absolute date range.
    """
    # Compute timelimit from the requested span, if both dates provided
    timelimit = None
    if date_start_iso and date_end_iso:
        start_utc = _to_utc(_parse_dt(date_start_iso))
        end_utc   = _to_utc(_parse_dt(date_end_iso))
        if start_utc and end_utc and end_utc >= start_utc:
            timelimit = _pick_timelimit_for_range(start_utc, end_utc)  # -> 'd' | 'w' | 'm' | 'y' | None

    # Execute DDG with timelimit hint only (no post-filtering)
    try:
        with DDGS() as ddgs:
            kwargs = dict(max_results=max_results)
            if timelimit:
                kwargs["timelimit"] = timelimit  # {'d','w','m','y'}
            results = list(ddgs.text(query, **kwargs))
    except Exception as e:
        print(f"[Search Error] '{query}' (topic='{subject_obj.get('topic','?')}'): {e}")
        return []

    # Return URLs as-is (no extra filtering)
    return [r.get("href") for r in results if r.get("href")]


# -------------------------------------------
# 6️⃣ Rephrase & Classify (using PAGE METADATA)
# -------------------------------------------
def rephrase_subject(subject_text: str, n_variants: int = 10) -> List[str]:
    messages = [
        {"role": "system", "content": "You only output JSON—no extra text."},
        {"role": "user", "content": (
            f"Rephrase this into {n_variants} search queries under 12 words."
            "Return ONLY a JSON list of strings."
            f"Subject: {subject_text}JSON:"
        )}
    ]
    raw = chat_generate_json(messages, max_new_tokens=384)
    return parse_json_list(raw) or [subject_text]

def classify_pages(pages: List[Dict[str, Any]], subject_obj: Dict[str, Any], debug: bool = False) -> Dict[str, Dict[str, Any]]:
    """
    Classify pages (with metadata) instead of bare URLs.
    Returns {url: {"category": int, "reason": str, "relevance": int}}
    - Enforces allowed category IDs by instruction; non-allowed items should be omitted by the model.
    - Post-filter still drops anything outside allowed_ids to be safe.
    - Robust JSON parsing (cleans trailing commas, code fences; accepts dict or list forms).
    """
    if not pages:
        return {}

    subject_text = subject_obj.get("topic", "")
    categories   = subject_obj.get("categories", []) or []
    allowed_ids  = {str(c["id"]) for c in categories} if categories else {"1", "2", "3"}
    allowed_ids_str = ", ".join(sorted(allowed_ids))
    cats_line = ", ".join(f"{c['id']}={c['name']}" for c in categories) if categories else (
        "1=Dashboard (interactive visualization), 2=Paid marketplace, 3=Downloadable dataset (CSV/JSON/ZIP)"
    )

    # ---- Compact payload to keep prompts small
    # - cap lengths
    # - take at most a few internal links per page
    # - drop entries with empty URL
    compact_pages = []
    for p in pages:
        u = (p.get("url") or "").strip()
        if not u:
            continue
        links = p.get("links") or []
        links_small = []
        for l in links[:6]:  # cap to 6 internal links
            href = (l.get("href") or "")[:200]
            text = (l.get("text") or "")[:80]
            if href:
                links_small.append({"href": href, "text": text})
        compact_pages.append({
            "url": u,
            "title": (p.get("title", "") or "")[:160],
            "desc": (p.get("description", "") or "")[:220],
            "h1":   (p.get("h1", "") or "")[:120],
            "links": links_small,
        })

    # Nothing to classify
    if not compact_pages:
        return {}

    payload_json = json.dumps(compact_pages, ensure_ascii=False)

    # ---- Strong instruction to restrict to allowed IDs and omit others
    messages = [
        {"role": "system", "content": "You only output JSON—no extra text."},
        {"role": "user", "content": (
            "Classify these PAGES (with metadata) about: "
            f"'{subject_text}'. "
            f"Categories: {cats_line}. "
            f"Allowed category IDs: [{allowed_ids_str}]. "
            "USE ONLY the allowed IDs. If a page does NOT match an allowed category, OMIT it entirely (do not include the URL). "
            "Output ONE JSON object ONLY (no wrapper keys). "
            'Keys MUST be the page "url" values. '
            'Values MUST be: {"category": <allowed id>, "reason":"short", "relevance": 1-5 (int, 5=high)}. '
            f"Pages JSON: {payload_json}\n"
            "JSON only:"
        )}
    ]

    # Generate JSON (deterministic)
    raw = chat_generate_json(messages, max_new_tokens=1024)
    if debug:
        preview = raw[:500] + ("…" if len(raw) > 500 else "")
        print("🧪 Raw model JSON:", preview)

    # --- Forgiving parse (accept dict OR list; cleanup trailing commas / code fences)
    def _cleanup_json_maybe(text: str) -> str:
        if not text:
            return text
        t = text.strip().replace("```json", "").replace("```", "")
        # remove trailing commas before ] or }
        t = re.sub(r",(\s*[\]\}])", r"\1", t)
        return t

    def _extract_obj_or_list(text: str) -> Optional[Any]:
        if not text:
            return None
        t = _cleanup_json_maybe(text)
        # 1) try whole string
        try:
            data = json.loads(t)
            if isinstance(data, (dict, list)):
                return data
        except Exception:
            pass
        # 2) find first JSON block
        first = next((c for c in t if c in "{}[]"), "")
        if not first:
            return None
        patt = r"\{.*\}" if first == "{" else r"\[.*\]"
        m = re.search(patt, t, flags=re.DOTALL)
        if m:
            try:
                block = _cleanup_json_maybe(m.group(0))
                data = json.loads(block)
                if isinstance(data, (dict, list)):
                    return data
            except Exception:
                return None
        return None

    parsed = _extract_obj_or_list(raw) or {}

    # Normalize common shapes: list-of-objects or {"URLs": {...}}
    if isinstance(parsed, dict) and "URLs" in parsed and isinstance(parsed["URLs"], dict):
        parsed = parsed["URLs"]
    elif isinstance(parsed, list):
        merged = {}
        for item in parsed:
            if isinstance(item, dict) and "url" in item:
                url_key = item.get("url")
                if url_key:
                    merged[url_key] = {
                        "category": item.get("category"),
                        "reason": item.get("reason", ""),
                        "relevance": item.get("relevance", 1),
                    }
            elif isinstance(item, dict) and len(item) == 1:
                # shape: [{"https://…": {...}}, {"https://…": {...}}]
                merged.update(item)
        parsed = merged

    if not isinstance(parsed, dict):
        if debug:
            print("⚠️ Parsed JSON is not an object after normalization. Got:", type(parsed).__name__)
        return {}

    # ---- Final sanitize + enforce allowed_ids
    out: Dict[str, Dict[str, Any]] = {}
    dropped = {"not_dict": 0, "no_category": 0, "bad_category": 0}

    for u, d in parsed.items():
        if not isinstance(d, dict):
            dropped["not_dict"] += 1
            continue

        # category → normalize to string int
        cat = d.get("category")
        try:
            cat_str = str(int(cat))
        except Exception:
            m = re.search(r"\d+", str(cat) if cat is not None else "")
            cat_str = m.group(0) if m else None

        if not cat_str:
            dropped["no_category"] += 1
            continue
        if cat_str not in allowed_ids:
            dropped["bad_category"] += 1
            continue  # enforce allowed IDs

        # relevance → int in [1..5]
        rel = d.get("relevance", 1)
        try:
            relevance = int(rel)
            if relevance < 1:
                relevance = 1
            elif relevance > 5:
                relevance = 5
        except Exception:
            relevance = 1

        # reason (short)
        reason = (d.get("reason", "") or "")[:180]

        out[u] = {"category": int(cat_str), "reason": reason, "relevance": relevance}

    if debug:
        print(f"✅ Kept {len(out)} items (dropped: {dropped}) | allowed_ids={sorted(allowed_ids)}")
    return out


# -------------------------------------------
# 7️⃣ DeepSearch Main (metadata + date-aware DDG → page-level classification)
#    - Dates remain whatever user set; NOT modified here
# -------------------------------------------
results: List[Dict[str, Any]] = []
print("🚀 Starting Gemma 3 DeepSearch…")

for subject in tqdm(subjects, desc="📌 Subjects", unit="subject"):
    topic = subject["topic"]
    n_variants = subject["nb of research deviations"]
    mr = subject.get("max_results", 20)

    print(f"\n🔍 Processing subject: {topic}")
    print(f"   → Variants: {n_variants}, Max results per query: {mr}")

    # Rephrase subject
    print("✏️ Rephrasing queries...")
    queries = rephrase_subject(topic, n_variants)
    print(f"   → Generated {len(queries)} rephrased queries")

    # Gather URLs via DDG (DATE-AWARE)
    all_urls, query_map = set(), {}

    print("🌐 Running DuckDuckGo searches (date-aware)…")
    blacklist_urls = set(links_df['url'].dropna().astype(str))
    for q in tqdm(queries, desc="   🔎 Queries", unit="query"):
        print(f"     → Searching for: '{q}'")
    
        # We will fetch more results than needed to have backups.
        # A factor of 3 is usually a safe bet.
        fetch_limit = mr * 3
    
        potential_urls = search_ddg(
            subject,
            q,
            max_results=fetch_limit, # Fetch a larger batch of URLs
            date_start_iso=subject.get("date research start"),
            date_end_iso=subject.get("date research end"),
        )
    
        print(f"       Found {len(potential_urls)} potential URLs to check against blacklist.")
    
        new_urls_added_count = 0
        for u in potential_urls:
            # Stop once we have found enough valid URLs for this query
            if new_urls_added_count >= mr:
                break
    
            # Check if the URL is new AND not on the blacklist
            if u not in all_urls and u not in blacklist_urls:
                all_urls.add(u)
                query_map[u] = q
                new_urls_added_count += 1
    
        print(f"       Added {new_urls_added_count} new, non-blacklisted URLs for this query.")
        time.sleep(0.4) # be polite

    subject["nb of results"] = mr
    print(f"📊 Total unique URLs gathered: {len(all_urls)}")

    if not all_urls:
        print("⚠️ No URLs found for this subject, skipping…")
        continue

    # Cleanup
    torch.cuda.empty_cache()
    gc.collect()

    # --- Fetch lightweight metadata for each URL ---
    print("🧩 Fetching page metadata…")
    unique_urls = list(all_urls)
    page_meta_map: Dict[str, Dict[str, Any]] = {}
    for u in tqdm(unique_urls, desc="   🌐 Pages", unit="page"):
        page_meta_map[u] = fetch_page_metadata(u)

    # Classify using page metadata
    print("🗂️ Starting classification of pages...")
    all_classified = {}
    batch_size = 5
    for i in tqdm(range(0, len(unique_urls), batch_size), desc="   📦 Classification batches", unit="batch"):
        chunk_urls = unique_urls[i:i+batch_size]
        # Build chunk of pages with metadata
        chunk_pages = []
        for u in chunk_urls:
            meta = page_meta_map.get(u, {}) or {}
            chunk_pages.append({
                "url": u,
                "title": meta.get("title", ""),
                "description": meta.get("description", ""),
                "h1": meta.get("h1", ""),
                "links": meta.get("links", []),
            })
        # Keep debug=True so you can see the output
        chunk_res = classify_pages(chunk_pages, subject, debug=True) 
        all_classified.update(chunk_res)
        time.sleep(1)

    # Aggregate
    for url, d in all_classified.items():
        results.append({
            "subject": topic,
            "query": query_map.get(url, ""),
            "url": url,
            "class": d["category"],
            "reason": d["reason"],
            "relevance": d.get("relevance", 1), # <-- NEW: Add relevance score
        })

    print(f"✅ Finished subject: {topic} ({len(all_classified)} classified URLs)")

# --- MODIFIED: Replaced the entire saving block ---

# -------------------------------------------
# 8️⃣ Filter, Rank, and Save Results
# -------------------------------------------
print("\n💾 Filtering and saving results...")
if results:
    # --- NEW: Filter to top X per subject based on relevance ---
    print(f"🔬 Found {len(results)} total classified items. Filtering to top-k per subject...")

    # 1. Create the mapping from subject topic -> max_final_results
    topic_to_max_final = {
        s['topic']: s.get('max_final_results', 10) for s in subjects # Default 10
    }

    # 2. Convert all results to DataFrame
    df = pd.DataFrame(results)
    final_dfs = []

    # 3. Group by subject, sort by relevance, and take the top-k
    for subject_topic, group_df in df.groupby('subject'):
        limit = topic_to_max_final.get(subject_topic, 10) # Get k
        
        # Sort by relevance (desc) and take the top 'limit' rows
        top_group_df = group_df.sort_values(by='relevance', ascending=False).head(limit)
        final_dfs.append(top_group_df)

    # 4. Combine the filtered groups back into one DataFrame
    if final_dfs:
        final_df = pd.concat(final_dfs).sort_index() # sort_index to restore original-ish order
        # Convert the final DF back to a list of dicts for JSON saving
        final_results_list = final_df.to_dict('records')
        print(f"✅ Filtered down to {len(final_results_list)} items.")
    else:
        print("⚠️ No items remained after grouping/filtering.")
        final_df = pd.DataFrame(columns=results[0].keys() if results else []) # empty DF
        final_results_list = []

    # 5. Save the *FILTERED* results
    final_df.to_csv("deepsearch_results.csv", index=False)
    with open("deepsearch_results.json", "w", encoding="utf-8") as f:
        json.dump(final_results_list, f, indent=2, ensure_ascii=False)

    # Save the subjects summary (which now includes 'max_final_results')
    pd.DataFrame(subjects).to_csv("deepsearch_subjects_summary.csv", index=False)
    print("🎉 Done! Filtered results saved to deepsearch_results.csv / .json and deepsearch_subjects_summary.csv")
else:
    print("🤷 No relevant results found.")

In [ ]:
from datetime import datetime
import pytz
import csv

paris = pytz.timezone('Europe/Paris')
today_str = datetime.now(paris).strftime('%d/%m/%Y')

# ADD result to GSHEET
data_ws = worksheet.spreadsheet.get_worksheet(2)

# Ensure headers exist on the 3rd tab (optional safety)
existing_header = data_ws.row_values(1)
expected_header = ["Date", "subject", "query", "url", "class", "reason", "relevance"]
if not existing_header:
    data_ws.insert_row(expected_header, index=1)

# Read CSV and build rows with today's date in DD/MM/YYYY
rows_to_append = []
csv_path = "/kaggle/working/deepsearch_results.csv"

with open(csv_path, newline='', encoding='utf-8') as f:
    reader = csv.DictReader(f)
    for r in reader:
        rows_to_append.append([
            today_str,
            r.get('subject', ''),
            r.get('query', ''),
            r.get('url', ''),
            # Coerce numeric-looking fields safely; leave blank if not present
            int(r['class']) if (r.get('class') or '').isdigit() else r.get('class', ''),
            r.get('reason', ''),
            int(r['relevance']) if (r.get('relevance') or '').isdigit() else r.get('relevance', ''),
        ])

# Batch-append (fast + preserves existing data)
if rows_to_append:
    data_ws.append_rows(rows_to_append, value_input_option='USER_ENTERED')

In [ ]:
import json
from datetime import datetime
import os
from urllib.parse import urlsplit
from collections import defaultdict

# --- Configuration ---
TEMPLATE_HTML = "/kaggle/input/template-pptx/newsletter_template.html"
JSON_DATA     = "/kaggle/working/deepsearch_results.json"
OUTPUT_HTML   = f"newsletter_{datetime.now().strftime('%Y-%m-%d')}.html"

# Placeholders from the template:
PH_DATE      = "<!-- DATE_PLACEHOLDER -->"
PH_SUMMARY   = "<!-- SUMMARY_ITEMS_PLACEHOLDER -->"
PH_SECTIONS  = "<!-- SUBJECT_SECTIONS_PLACEHOLDER -->"

# --- 1) Load JSON robustly (array JSON or NDJSON), avoid crashing on empty ---
def load_results(path):
    try:
        with open(path, "r", encoding="utf-8") as f:
            raw = f.read().strip()
            if not raw:
                # Empty file -> return empty list
                return []
            # Try standard JSON first
            try:
                obj = json.loads(raw)
                if isinstance(obj, list):
                    return obj
                # If it's a dict, try to find a list under a common key
                for k in ("results", "data", "items"):
                    if isinstance(obj.get(k, None), list):
                        return obj[k]
                # Fallback: wrap single object
                return [obj]
            except json.JSONDecodeError:
                # Try NDJSON line-by-line
                items = []
                for line in raw.splitlines():
                    line = line.strip()
                    if not line:
                        continue
                    try:
                        items.append(json.loads(line))
                    except json.JSONDecodeError:
                        # Skip bad lines rather than crashing
                        continue
                return items
    except FileNotFoundError:
        print(f"Erreur : Le fichier JSON '{path}' est introuvable.")
        raise SystemExit(1)

data = load_results(JSON_DATA)

# --- 2) Group by subject (keep all results) ---
subjects_data = defaultdict(list)
for item in data:
    subject = (item or {}).get("subject")
    if not subject:
        continue
    subjects_data[subject].append({
        "url": item.get("url", "N/A"),
        "reason": item.get("reason", "N/A"),
        "class_id": item.get("class", 0),
    })
print(f"{len(subjects_data)} sujets trouvés dans le JSON.")

# --- 3A) Build the summary (max 5 subjects) ---
summary_parts = []
for i, subject in enumerate(list(subjects_data.keys())[:5]):
    summary_parts.append(f"""
    <tr>
        <td style="padding-bottom: 10px;">
            <table align="center">
                <tr>
                    <td align="center" valign="middle" style="padding:0;">
                      <div style="
                          background-color:#e84379;
                          border-radius:50%;
                          width:30px;
                          height:30px;
                          display:inline-block;
                          text-align:center;
                          line-height:30px;
                      ">
                        <p style="
                            color:#ffffff;
                            font-size:16px;
                            font-weight:bold;
                            margin:0;
                            line-height:30px;
                        ">{i+1}</p>
                      </div>
                    </td>
                    <td style="padding-left: 15px;">
                        <p style="color: #ffffff; font-size: 16px; margin: 0; text-align: left;">{subject}</p>
                    </td>
                </tr>
            </table>
        </td>
    </tr>""")
summary_html = "".join(summary_parts)
print("Sommaire HTML généré.")

# --- 3B) Build subject sections ---
section_blocks = []
for i, (subject, links) in enumerate(subjects_data.items()):
    bg_color = "#ffffff" if i % 2 == 0 else "#F8F9FA"

    link_lines = []
    for link in links:
        url = link["url"]
        netloc = urlsplit(url).netloc if url and url != "N/A" else "N/A"
        link_lines.append(f"""
        <p style="font-size: 12px; color: #333333; margin-bottom: 15px; word-break: break-all;">
            - <a target="_blank" href="{url}" style="color: #0066cc;">{netloc}</a> &rarr; {link['reason']}
        </p>""")
    links_html = "".join(link_lines)

    section_blocks.append(f"""
    <tr>
        <td style="padding: 0 0 40px; background-color: {bg_color};">
            <img src="https://i.imgur.com/CLeuYYS.png" alt="divider" style="width:100%; max-width:600px; display: block; margin-bottom: 40px;">
            <div style="padding: 0 30px;">
                <p style="color: #888888; font-size: 14px; margin: 0;">Newsletter list</p>
                <p style="color: #1a1a1a; font-size: 24px; font-weight: bold; margin: 5px 0 20px;">{subject}</p>
                {links_html}
            </div>
        </td>
    </tr>""")
sections_html = "".join(section_blocks)
print("Sections de contenu HTML générées.")

# --- 4) Read template and substitute the 3 placeholders ---
try:
    with open(TEMPLATE_HTML, "r", encoding="utf-8") as f:
        template_content = f.read()
except FileNotFoundError:
    print(f"Erreur : Le fichier template '{TEMPLATE_HTML}' est introuvable.")
    raise SystemExit(1)

final_html = (
    template_content
        .replace(PH_DATE, datetime.now().strftime("%d/%m/%Y"))
        .replace(PH_SUMMARY, summary_html)
        .replace(PH_SECTIONS, sections_html)
)

# --- 5) Save the final HTML ---
with open(OUTPUT_HTML, "w", encoding="utf-8") as f:
    f.write(final_html)

print(f"\nNewsletter générée avec succès ! Fichier sauvegardé sous : '{os.path.abspath(OUTPUT_HTML)}'")


In [ ]:
# Get email
worksheet = spreadsheet.get_worksheet(1) # To get the third tab

# --- Fetch all data from the worksheet ---
print("Fetching all records from the worksheet...")
data = worksheet.get_all_records()


emails_df = pd.DataFrame(data)
email_list = [email.strip() for email in emails_df['List of contacts'][0].split(',')]

In [ ]:
!pip install sib_api_v3_sdk

In [ ]:
import sib_api_v3_sdk
from sib_api_v3_sdk.rest import ApiException
import os
from datetime import datetime

# --- 1. Configuration ---

# Paste your Brevo API key here
API_KEY = user_secrets.get_secret("BREVO") 

# This is the HTML file you just created
# It uses the variable from your previous script
HTML_TO_SEND = OUTPUT_HTML # Or "newsletter_2025-10-21.html"

# --- Email details ---
SENDER_EMAIL = "menardisaac@gmail.com"
SENDER_NAME = "Newsletter MyTraffic Bot"
EMAIL_SUBJECT = f"Your Newsletter - {datetime.now().strftime('%Y-%m-%d')}"

unique_emails = email_list
print(f"Preparing to send email to {len(unique_emails)} unique recipients: {unique_emails}")


# --- 2. Read the HTML file content ---
try:
    with open(HTML_TO_SEND, 'r', encoding='utf-8') as f:
        html_content = f.read()
    print(f"Successfully read HTML file: {HTML_TO_SEND}")
except FileNotFoundError:
    print(f"ERROR: HTML file not found at {HTML_TO_SEND}")
    # exit() # Uncomment if you want to stop the script on error
except Exception as e:
    print(f"ERROR: Could not read HTML file: {e}")
    # exit() # Uncomment if you want to stop the script on error

# --- 3. Configure Brevo API ---
configuration = sib_api_v3_sdk.Configuration()
configuration.api_key['api-key'] = API_KEY

api_instance = sib_api_v3_sdk.TransactionalEmailsApi(sib_api_v3_sdk.ApiClient(configuration))

# --- 4. Create the Email Object ---
sender = {"name": SENDER_NAME, "email": SENDER_EMAIL}

# --- MODIFIED: Build the 'to' list for the API ---
# The API expects a list of dictionaries, e.g., [{'email':'a@b.com'}, {'email':'c@d.com'}]
to = [{"email": email} for email in unique_emails]


send_smtp_email = sib_api_v3_sdk.SendSmtpEmail(
    to=to,
    sender=sender,
    subject=EMAIL_SUBJECT,
    html_content=html_content
)

# --- 5. Send the Email ---
try:
    api_response = api_instance.send_transac_email(send_smtp_email)
    print("\n✅ Email sent successfully!")
    print("Brevo API Response:")
    print(api_response)
except ApiException as e:
    print(f"\n❌ Error sending email via Brevo: {e}")